In [1]:
# Initialize Jupyter Notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"
%matplotlib inline
# from matplotlib import pyplot as plt
# plt.rcParams["font.sans-serif"] = ["SimHei"]

In [2]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

In [18]:
_from = "深圳北"
_to   = "广州南"
_date = "2022-05-26"

In [20]:
driver =  webdriver.Chrome()
driver.set_window_size(900,500)
# bypass selenium detection
driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument",
                        {"source": """Object.defineProperty(navigator, 'webdriver', {get: () => undefined})"""})
# visit
driver.get("https://kyfw.12306.cn/otn/resources/login.html")
driver.find_element(by=By.CSS_SELECTOR,value="#J-userName").send_keys(your_account)
time.sleep(1)
driver.find_element(by=By.CSS_SELECTOR,value="#J-password").send_keys(your_password)
time.sleep(1)
driver.find_element(by=By.CSS_SELECTOR,value="#J-login").click()
time.sleep(2)
source = driver.find_element(by=By.CSS_SELECTOR,value="#nc_1_n1z")
target = driver.find_element(by=By.CSS_SELECTOR,value="#J-password")
time.sleep(2)
# There is a login sliding authentication window.
# It is the reason why I import `ActionChains`.
# `duration` means how long, the unit of it is `ms`.
# `duration=1500` means selenium will spend 1.5s on drag it. 
# It would be better to not set it too low.
# After my practice, the setting of 1.5 seconds is the most robust.
ActionChains(driver,duration=1500).drag_and_drop(source, target).perform()
try:
    time.sleep(2)
    driver.find_element(by=By.CSS_SELECTOR,value="#nc_1_refresh1").click()
    time.sleep(5)
    print("You have to drag **BY YOURSELF**")
    ActionChains(driver,duration=3000).drag_and_drop(source, target).perform()
except:
    time.sleep(1)
driver.implicitly_wait(10)
driver.find_element(by=By.CSS_SELECTOR,value="div.modal-ft > a").click()
hover = driver.find_element(by=By.CSS_SELECTOR,value="#J-chepiao > a")
ActionChains(driver,duration=2).move_to_element(hover).perform()
driver.find_element(by=By.CSS_SELECTOR,value="#megamenu-3 > div:nth-child(1) > ul > li.nav_dan > a").click()
time.sleep(2)
driver.find_element(by=By.CSS_SELECTOR,value="#qd_closeDefaultWarningWindowDialog_id").click()
time.sleep(2)
driver.find_element(by=By.CSS_SELECTOR,value="#fromStationText").click()
driver.find_element(by=By.CSS_SELECTOR,value="#fromStationText").send_keys(_from)
driver.find_element(by=By.CSS_SELECTOR,value="#fromStationText").send_keys(Keys.ENTER)
driver.find_element(by=By.CSS_SELECTOR,value="#toStationText").click()
driver.find_element(by=By.CSS_SELECTOR,value="#toStationText").send_keys(_to)
driver.find_element(by=By.CSS_SELECTOR,value="#toStationText").send_keys(Keys.ENTER)
driver.find_element(by=By.CSS_SELECTOR,value="#train_date").clear()
driver.find_element(by=By.CSS_SELECTOR,value="#train_date").click()
driver.find_element(by=By.CSS_SELECTOR,value="#train_date").send_keys(_date)
driver.find_element(by=By.CSS_SELECTOR,value="#train_date").send_keys(Keys.ENTER)
time.sleep(1)
driver.find_element(by=By.CSS_SELECTOR,value="#query_ticket").click()
time.sleep(2)
driver.find_element(by=By.CSS_SELECTOR,value="#cc_from_station_{}_check".format(_from)).click()
time.sleep(2)
driver.implicitly_wait(10)
# Not every train can be booked, and some may not be booked because they are sold out.
book = driver.find_elements(by=By.CSS_SELECTOR,value="td.no-br")
bookable = driver.find_elements(by=By.XPATH,value="//a[@class='btn72']/parent::td")
book_id_l = [i.id for i in book]
bookable_id_l = [i.id for i in bookable]
# Find tickets that can't be booked, which means I won't have to keep an eye on it
non_bookable_index = [book_id_l.index(i) for i in book_id_l if i not in bookable_id_l]

# leave time and reach time
leave_t_element = driver.find_elements(by=By.CSS_SELECTOR,value="div.cds > strong.start-t")
reach_t_element = driver.find_elements(by=By.CSS_SELECTOR,value="div.cds > strong.color999")
for i in non_bookable_index:
    leave_t_element[i] = None
    reach_t_element[i] = None
leave_t_element = [i for i in leave_t_element if i != None]
reach_t_element = [i for i in reach_t_element if i != None]

# The two lists should be the same length.
# If this is not the case, then it means that something might be something wrong.
# Output them is just to verify.
print(len(leave_t_element))
print(len(reach_t_element))

leave_t = [time.strptime(str(_date)+" "+i.text,"%Y-%m-%d %H:%M") for i in leave_t_element]
reach_t = [time.strptime(str(_date)+" "+i.text,"%Y-%m-%d %H:%M") for i in reach_t_element]
ind_record = []
decision_leave = []
decision_reach = []
for i in leave_t:
    if int(i.tm_hour) >= 9 and int(i.tm_hour) < 10:
        ind = leave_t.index(i)
        decision_leave.append(leave_t[ind])
        decision_reach.append(reach_t[ind])
        ind_record.append(ind)
print("decision_leave",[time.strftime("%Y-%m-%d %H:%M",i) for i in decision_leave])
print("decision_reach",[time.strftime("%Y-%m-%d %H:%M",i) for i in decision_reach])
final_leave = input("Choose `decision_leave` (copy and paste will be okay)")
book[leave_t.index(time.strptime(final_leave,"%Y-%m-%d %H:%M"))].click()
time.sleep(1)
passengers = driver.find_elements(by=By.CSS_SELECTOR,value="#normal_passenger_id > li > label")
names = [i.text for i in passengers]
print("names",names)
passenger = input("Choose `names` (copy and paste will be okay)")
# check that person
driver.find_element(by=By.CSS_SELECTOR,value="#normal_passenger_id > li:nth-child({}) > label".format(names.index(passenger)+1)).click()
time.sleep(1)
driver.find_element(by=By.CSS_SELECTOR,value="#dialog_xsertcj_cancel").click()
time.sleep(1)
driver.find_element(by=By.CSS_SELECTOR,value="#submitOrder_id").click()
time.sleep(1)
driver.find_element(by=By.XPATH,value="/html/body/div[6]/div/div[5]/div[1]/div/div[2]/div[2]/div[3]/div[2]/div[2]/ul[2]/li[2]/a").click()
time.sleep(1)

# This last line of code is to submit the order, if not commented out then you will truely submit the order.
# If you are checking or debugging the code, I suggest you comment it out.😘
# driver.find_element(by=By.CSS_SELECTOR,value="#qr_submit_id").click()

{'identifier': '2'}

114
114
decision_leave ['2022-05-26 09:02', '2022-05-26 09:13', '2022-05-26 09:19', '2022-05-26 09:21', '2022-05-26 09:25', '2022-05-26 09:29', '2022-05-26 09:30', '2022-05-26 09:35', '2022-05-26 09:50']
decision_reach ['2022-05-26 09:39', '2022-05-26 09:50', '2022-05-26 09:55', '2022-05-26 10:58', '2022-05-26 10:01', '2022-05-26 10:41', '2022-05-26 10:06', '2022-05-26 10:11', '2022-05-26 10:26']


# Thank you for reading and have a great day!❤